# Prediction
After the exploratory data analysis, a prediction will be made.
The folowing steps for this prediction:
1. Preprocessing - Handle missing data, Feature engineering.
2. Predictions - predicting with validation data and then with test data, metrics.
## 📚

In [292]:
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

#pipeline tools
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from pipelinehelper import PipelineHelper
from Column_Selector import Selector

ImportError: cannot import name 'Selector' from 'Column_Selector' (C:\Users\97254\Projects\Home Equity Loan Defaut Prediction\Column_Selector.py)

## Baseline 1: Basic approach

### Decisions:
* **Preprocessing:**
    1. **Missing data:** **Categoric** - fill with most frequent. **Discrete** - fill with most frequent. **Continuous** - fill with median.               
    2. **Feature Engineering**: only categoric labels, label encoding.
* **model training** - using cross validation in validation data on a simple decition tree model.
* **model testing** - train on whole train + validation set and use test data for results.

In [288]:

cat_imp = SimpleImputer(strategy="most_frequent")

Categoric = FunctionTransformer(lambda x: x[Cats], validate=False)
Categoric_features = Pipeline([
                ('selector', Selector(Categoric)),
                ('imputer', cat_imp),
                ('LabelEncoder', LabelEncoder())
            ])

NameError: name 'Selector' is not defined

In [272]:
cont_imp = SimpleImputer(strategy="median")
Continuous = Continuous+["LOAN"]
Cont_FT = FunctionTransformer(lambda x: x[Continuous], validate=False)
Continuous_features = Pipeline([
                ('selector', Cont_FT),
                ('imputer', cont_imp)
            ])

In [273]:
disc_imp = SimpleImputer(strategy="most_frequent")

disc_FT = FunctionTransformer(lambda x: x[Discrete], validate=False)
Discrete_features = Pipeline([
                ('selector', disc_FT),
                ('imputer', disc_imp)
            ])

In [281]:
feats = FeatureUnion([('Categoric_features', Categoric_features),
                      ('Continuous_features',Continuous_features),
                      ('Discrete_features',Discrete_features)                      
                      ])



In [282]:
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

array([[6.23510000e+04, 8.46240000e+04, 8.81265168e+01, ...,
        0.00000000e+00, 1.00000000e+00, 2.20000000e+01],
       [3.89000000e+04, 6.05000000e+04, 9.59000000e+01, ...,
        4.00000000e+00, 2.00000000e+00, 5.30000000e+01],
       [6.19070000e+04, 8.71330000e+04, 8.14921194e+01, ...,
        1.00000000e+00, 0.00000000e+00, 2.20000000e+01],
       ...,
       [9.41650000e+04, 1.30285000e+05, 1.37813331e+02, ...,
        1.00000000e+00, 0.00000000e+00, 2.80000000e+01],
       [6.34750000e+04, 8.94300000e+04, 2.86301815e+02, ...,
        0.00000000e+00, 0.00000000e+00, 1.90000000e+01],
       [6.42950000e+04, 7.35940000e+04, 1.57246550e+02, ...,
        0.00000000e+00, 0.00000000e+00, 1.10000000e+01]])